In [1]:
import pickle
from copy import deepcopy
import time

# data prep and model-tuning
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# types of models we'll fit
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.multioutput import RegressorChain

In [2]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import stats

In [3]:
from warnings import simplefilter
simplefilter(action='ignore', category=UserWarning)

In [4]:
sns.set_style('darkgrid')
%matplotlib inline

## Our Data

In [5]:
PLOT_DATA = '../data/processed/plot_features.csv'
KEEP_PLOT_COLS = ['uuid', 'lat', 'lon', 'ecoregion3', 'agency', 'distance_to_water_m', 'plot_size_ac', 'meas_yr']
plot_data = pd.read_csv(PLOT_DATA)[KEEP_PLOT_COLS]
plot_data.columns = [col.upper() for col in plot_data.columns]
plot_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5089 entries, 0 to 5088
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   UUID                 5089 non-null   object 
 1   LAT                  5089 non-null   float64
 2   LON                  5089 non-null   float64
 3   ECOREGION3           5089 non-null   object 
 4   AGENCY               5089 non-null   object 
 5   DISTANCE_TO_WATER_M  5089 non-null   float64
 6   PLOT_SIZE_AC         5089 non-null   float64
 7   MEAS_YR              5089 non-null   int64  
dtypes: float64(4), int64(1), object(3)
memory usage: 318.2+ KB


In [6]:
LIDAR_DATA = '../data/processed/lidar_features.csv'
lidar_data = pd.read_csv(LIDAR_DATA)[['uuid', 'elevation', 'cover']].rename({'uuid': 'UUID', 'cover': 'lidar_cover'}, axis=1)
lidar_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5411 entries, 0 to 5410
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   UUID         5411 non-null   object 
 1   elevation    5411 non-null   float64
 2   lidar_cover  5411 non-null   float64
dtypes: float64(2), object(1)
memory usage: 126.9+ KB


In [7]:
plot_data = plot_data.merge(lidar_data, left_on=['UUID'], right_on=['UUID'], how='inner').drop_duplicates(subset=['UUID'])
plot_data.columns = [col.upper() for col in plot_data.columns]
plot_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4272 entries, 0 to 5410
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   UUID                 4272 non-null   object 
 1   LAT                  4272 non-null   float64
 2   LON                  4272 non-null   float64
 3   ECOREGION3           4272 non-null   object 
 4   AGENCY               4272 non-null   object 
 5   DISTANCE_TO_WATER_M  4272 non-null   float64
 6   PLOT_SIZE_AC         4272 non-null   float64
 7   MEAS_YR              4272 non-null   int64  
 8   ELEVATION            4272 non-null   float64
 9   LIDAR_COVER          4272 non-null   float64
dtypes: float64(6), int64(1), object(3)
memory usage: 367.1+ KB


In [8]:
INVENTORY = '../data/processed/inventory_features.csv'
inv_data = pd.read_csv(INVENTORY).rename({'uuid': 'UUID', 'year':'YEAR'}, axis=1).set_index(['UUID', 'YEAR'])
inv_data.columns = [col.upper() for col in inv_data.columns]
inv_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 49410 entries, ('ba510248', 2010) to ('c4f7f099', 2025)
Data columns (total 75 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TPA                  49410 non-null  int64  
 1   BA                   49410 non-null  int64  
 2   SDI                  49410 non-null  int64  
 3   CCF                  49410 non-null  int64  
 4   QMD                  49410 non-null  float64
 5   TCUFT                49410 non-null  int64  
 6   TOPHT                49410 non-null  int64  
 7   NUMBER_OF_STRATA     49410 non-null  int64  
 8   TOTAL_COVER          49410 non-null  int64  
 9   STRUCTURE_CLASS      49410 non-null  object 
 10  CANOPY_BASEHEIGHT    49410 non-null  int64  
 11  CANOPY_BULKDENSITY   49410 non-null  float64
 12  ABOVEGROUND_BIOMASS  49410 non-null  int64  
 13  ABOVEGROUND_CARBON   49410 non-null  int64  
 14  GS_TPA               49410 non-null  int64  
 15  AF    

In [9]:
SATELLITE = '../data/processed/satellite_features.csv'
sat = pd.read_csv(SATELLITE).rename({'uuid': 'UUID', 'year': 'YEAR'}, axis=1).set_index(['UUID', 'YEAR'])
S2_COLS = [col for col in sat.columns if col.startswith('S2')]
LANDTRENDR_COLS = [col for col in sat.columns if col.startswith('LT')]
sat = sat[S2_COLS + LANDTRENDR_COLS].dropna()
sat.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10178 entries, ('00027724', 2019) to ('fff7e1c3', 2020)
Data columns (total 53 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   S2_R_LEAFOFF           10178 non-null  float64
 1   S2_G_LEAFOFF           10178 non-null  float64
 2   S2_B_LEAFOFF           10178 non-null  float64
 3   S2_NIR_LEAFOFF         10178 non-null  float64
 4   S2_SWIR1_LEAFOFF       10178 non-null  float64
 5   S2_SWIR2_LEAFOFF       10178 non-null  float64
 6   S2_RE1_LEAFOFF         10178 non-null  float64
 7   S2_RE2_LEAFOFF         10178 non-null  float64
 8   S2_RE3_LEAFOFF         10178 non-null  float64
 9   S2_RE4_LEAFOFF         10178 non-null  float64
 10  S2_R_LEAFON            10178 non-null  float64
 11  S2_G_LEAFON            10178 non-null  float64
 12  S2_B_LEAFON            10178 non-null  float64
 13  S2_NIR_LEAFON          10178 non-null  float64
 14  S2_SWIR1_LEAFON        1

## Filter out some of the training data
We can exclude some of the training data based on how far separated the inventory data (interpolated using FVS simulations) is from the year the imagery was collected. Similarly, we can screen out training examples that had relatively low density of lidar returns.

In [10]:
sat_and_inv = sat.merge(inv_data, how='inner', left_index=True, right_index=True).reset_index()
sat_and_inv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9385 entries, 0 to 9384
Columns: 130 entries, UUID to LARCH
dtypes: float64(47), int64(81), object(2)
memory usage: 9.3+ MB


In [11]:
df = sat_and_inv.merge(plot_data, how='inner', left_on=['UUID'], right_on=['UUID']).dropna()
print('{:,d} samples'.format(len(df)))
print('Columns:', df.columns.values)

7,952 samples
Columns: ['UUID' 'YEAR' 'S2_R_LEAFOFF' 'S2_G_LEAFOFF' 'S2_B_LEAFOFF'
 'S2_NIR_LEAFOFF' 'S2_SWIR1_LEAFOFF' 'S2_SWIR2_LEAFOFF' 'S2_RE1_LEAFOFF'
 'S2_RE2_LEAFOFF' 'S2_RE3_LEAFOFF' 'S2_RE4_LEAFOFF' 'S2_R_LEAFON'
 'S2_G_LEAFON' 'S2_B_LEAFON' 'S2_NIR_LEAFON' 'S2_SWIR1_LEAFON'
 'S2_SWIR2_LEAFON' 'S2_RE1_LEAFON' 'S2_RE2_LEAFON' 'S2_RE3_LEAFON'
 'S2_RE4_LEAFON' 'S2_NDVI_LEAFON' 'S2_SAVI_LEAFON' 'S2_BRIGHTNESS_LEAFON'
 'S2_GREENNESS_LEAFON' 'S2_WETNESS_LEAFON' 'S2_NDVI_LEAFOFF'
 'S2_SAVI_LEAFOFF' 'S2_BRIGHTNESS_LEAFOFF' 'S2_GREENNESS_LEAFOFF'
 'S2_WETNESS_LEAFOFF' 'S2_dR' 'S2_dG' 'S2_dB' 'S2_dNIR' 'S2_dSWIR1'
 'S2_dSWIR2' 'S2_dRE1' 'S2_dRE2' 'S2_dNDVI' 'S2_dSAVI' 'S2_dBRIGHTNESS'
 'S2_dGREENNESS' 'S2_dWETNESS' 'S2_dRE3' 'S2_dRE4' 'LT_DUR_NBR'
 'LT_DUR_SWIR1' 'LT_MAG_NBR' 'LT_MAG_SWIR1' 'LT_RATE_NBR' 'LT_RATE_SWIR1'
 'LT_YSD_NBR' 'LT_YSD_SWIR1' 'TPA' 'BA' 'SDI' 'CCF' 'QMD' 'TCUFT' 'TOPHT'
 'NUMBER_OF_STRATA' 'TOTAL_COVER' 'STRUCTURE_CLASS' 'CANOPY_BASEHEIGHT'
 'CANOPY_BULKDENSITY' '

In [12]:
OUTLIERS = '../data/interim/outlier_uuids.csv'
outliers = pd.read_csv(OUTLIERS)
# filter out the height outliers
df = df[~df.UUID.isin(outliers.outlier_uuid)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7952 entries, 0 to 7951
Columns: 139 entries, UUID to LIDAR_COVER
dtypes: float64(53), int64(82), object(4)
memory usage: 8.5+ MB


In [13]:
df = df.loc[(df.TOPHT > 0)&(df.TOTAL_COVER >= 0)&(df.QMD > 0)]
# df = df.loc[(df.TOPHT > 0)&(df.LIDAR_COVER >= 10)&(df.QMD > 0)]
df.loc[df.QMD > 50, 'qmd'] = 50
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7708 entries, 0 to 7951
Columns: 140 entries, UUID to qmd
dtypes: float64(54), int64(82), object(4)
memory usage: 8.3+ MB


In [14]:
df.to_csv('../data/processed/sentinel_structure_training_data.csv', index=False, header=True)

## Inspect how many samples we have for different years, regions, etc.

In [15]:
df.groupby(by=['YEAR'])[['UUID']].count().rename({'uuid':'count'}, axis=1)

,UUID
YEAR,
2019,4099
2020,3609


In [16]:
pd.pivot_table(df, 
               values='UUID', 
               aggfunc='count', 
               index=['MEAS_YR'], 
               columns=['YEAR'], 
               fill_value=0)

YEAR,2019,2020
MEAS_YR,,
2010,490,0
2011,392,392
2013,752,752
2014,459,459
2015,491,491
2016,674,674
2017,499,499
2018,342,342


In [17]:
ecoreg_counts = df.groupby(by=['ECOREGION3'])[['UUID', 'YEAR', 'PLOT_SIZE_AC']].nunique()
ecoreg_counts

,UUID,YEAR,PLOT_SIZE_AC
ECOREGION3,,,
blue_mountains,110,2,1
cascades,713,2,3
coast_range,1771,2,2
columbia_plateau,13,2,1
eastern_cascades_slopes_and_foothills,434,2,2
klamath_mountains_california_high_north_coast_range,296,2,2
north_cascades,432,2,2
northern_rockies,122,2,1
puget_lowland,163,2,1


## Available features
The different types of predictor variables we can use to predict a forest attribute, including climate, lidar-derived, soil, and satellite imagery.

In [18]:
df[S2_COLS + LANDTRENDR_COLS].describe()

,S2_R_LEAFOFF,S2_G_LEAFOFF,S2_B_LEAFOFF,S2_NIR_LEAFOFF,S2_SWIR1_LEAFOFF,S2_SWIR2_LEAFOFF,S2_RE1_LEAFOFF,S2_RE2_LEAFOFF,S2_RE3_LEAFOFF,S2_RE4_LEAFOFF,...,S2_dRE3,S2_dRE4,LT_DUR_NBR,LT_DUR_SWIR1,LT_MAG_NBR,LT_MAG_SWIR1,LT_RATE_NBR,LT_RATE_SWIR1,LT_YSD_NBR,LT_YSD_SWIR1
count,7708.000000,7708.000000,7708.000000,7708.000000,7708.000000,7708.000000,7708.000000,7708.000000,7708.000000,7708.000000,...,7708.000000,7708.000000,7708.000000,7708.000000,7708.000000,7708.000000,7708.000000,7708.000000,7708.000000,7708.000000
mean,606.220706,680.174695,583.685829,2284.744952,712.242937,382.819250,930.007975,1876.339342,2130.404596,2290.689006,...,360.229905,417.447696,18.242086,14.270498,80.029190,292.274131,17.226518,319.817462,28.578620,26.555008
std,1046.496651,1051.096740,1078.714105,958.400257,314.879348,231.979016,1045.904615,984.708696,933.465047,883.204884,...,1021.978121,956.320125,12.275768,14.925823,193.113362,639.630613,36.187498,530.250292,10.468631,9.956267
min,9.797190,26.922216,1.000000,348.355904,91.010674,42.869246,70.049804,257.344712,311.294348,379.008394,...,-6641.178021,-6084.150834,1.000000,1.000000,-734.000000,-3098.000000,-116.000000,-390.000000,0.000000,0.000000
25%,138.312292,226.456214,121.474867,1702.408017,524.966903,248.190720,437.145189,1334.535597,1579.458543,1748.899616,...,145.159584,161.469546,6.000000,1.000000,-4.000000,-125.250000,0.000000,-5.000000,27.000000,21.000000
50%,218.161288,312.829155,195.150466,2112.570389,651.075741,312.776245,574.204674,1636.446904,1938.376937,2133.245012,...,475.905524,495.082222,19.000000,4.000000,42.000000,157.000000,3.000000,33.000000,34.000000,31.000000
75%,546.130330,575.580973,462.785578,2623.662855,815.419255,431.150948,906.008269,2079.902223,2414.413281,2614.498213,...,824.124349,862.869615,27.000000,35.000000,136.000000,553.000000,18.000000,423.000000,35.000000,34.000000
max,9046.410985,9303.687276,9492.814974,9533.337612,3020.551859,2062.405424,9169.860337,9173.494865,9085.315148,8913.603338,...,4035.860442,4245.146064,36.000000,36.000000,1071.000000,3133.000000,268.000000,3133.000000,35.000000,35.000000


## Selecting features and targets
This is the first step in determining what features we want to use, and what we want to predict.

In [19]:
X_COLS = S2_COLS + LANDTRENDR_COLS + ['ELEVATION', 'LAT', 'LON'] + ['ECOREGION3'] 
Y_COLS = ['LIDAR_COVER', 'TOPHT', 'QMD', 'SDI', 'TCUFT', 'ABOVEGROUND_BIOMASS']

Y_NAMES = [col.upper() for col in Y_COLS]

In [20]:
df[Y_COLS].describe()

,LIDAR_COVER,TOPHT,QMD,SDI,TCUFT,ABOVEGROUND_BIOMASS
count,7708.000000,7708.000000,7708.000000,7708.000000,7708.000000,7708.000000
mean,66.853895,96.780747,14.296492,315.817592,7410.013882,181.553321
std,30.918710,39.322274,8.100335,177.210972,6312.855165,121.967801
min,0.000000,6.000000,1.427506,2.000000,1.000000,2.000000
25%,49.677923,68.000000,8.730670,181.000000,2579.750000,88.000000
50%,80.035060,94.000000,12.820993,310.000000,5871.500000,165.000000
75%,91.370873,123.000000,17.797910,435.000000,10565.000000,246.000000
max,100.000000,267.000000,82.039021,1307.000000,51330.000000,887.000000


In [21]:
X_COLS

['S2_R_LEAFOFF',
 'S2_G_LEAFOFF',
 'S2_B_LEAFOFF',
 'S2_NIR_LEAFOFF',
 'S2_SWIR1_LEAFOFF',
 'S2_SWIR2_LEAFOFF',
 'S2_RE1_LEAFOFF',
 'S2_RE2_LEAFOFF',
 'S2_RE3_LEAFOFF',
 'S2_RE4_LEAFOFF',
 'S2_R_LEAFON',
 'S2_G_LEAFON',
 'S2_B_LEAFON',
 'S2_NIR_LEAFON',
 'S2_SWIR1_LEAFON',
 'S2_SWIR2_LEAFON',
 'S2_RE1_LEAFON',
 'S2_RE2_LEAFON',
 'S2_RE3_LEAFON',
 'S2_RE4_LEAFON',
 'S2_NDVI_LEAFON',
 'S2_SAVI_LEAFON',
 'S2_BRIGHTNESS_LEAFON',
 'S2_GREENNESS_LEAFON',
 'S2_WETNESS_LEAFON',
 'S2_NDVI_LEAFOFF',
 'S2_SAVI_LEAFOFF',
 'S2_BRIGHTNESS_LEAFOFF',
 'S2_GREENNESS_LEAFOFF',
 'S2_WETNESS_LEAFOFF',
 'S2_dR',
 'S2_dG',
 'S2_dB',
 'S2_dNIR',
 'S2_dSWIR1',
 'S2_dSWIR2',
 'S2_dRE1',
 'S2_dRE2',
 'S2_dNDVI',
 'S2_dSAVI',
 'S2_dBRIGHTNESS',
 'S2_dGREENNESS',
 'S2_dWETNESS',
 'S2_dRE3',
 'S2_dRE4',
 'LT_DUR_NBR',
 'LT_DUR_SWIR1',
 'LT_MAG_NBR',
 'LT_MAG_SWIR1',
 'LT_RATE_NBR',
 'LT_RATE_SWIR1',
 'LT_YSD_NBR',
 'LT_YSD_SWIR1',
 'ELEVATION',
 'LAT',
 'LON',
 'ECOREGION3']

In [22]:
USE_REGIONS = ['blue_mountains', 'coast_range', 'north_cascades', 'cascades',
               'klamath_mountains_california_high_north_coast_range', 
               'eastern_cascades_slopes_and_foothills', 'northern_rockies',
               'puget_lowland', 'willamette_valley']
display(df.groupby('ECOREGION3')[Y_COLS].mean().round(1).loc[USE_REGIONS])
display(df[Y_COLS].describe())

,LIDAR_COVER,TOPHT,QMD,SDI,TCUFT,ABOVEGROUND_BIOMASS
ECOREGION3,,,,,,
blue_mountains,33.0,59.5,13.2,108.3,1636.8,44.8
coast_range,73.2,110.2,16.6,363.6,9718.3,232.1
north_cascades,68.2,84.7,12.0,293.9,5827.7,153.8
cascades,75.0,102.8,13.8,346.4,7940.3,197.5
klamath_mountains_california_high_north_coast_range,72.8,93.6,11.3,371.4,7164.3,156.6
eastern_cascades_slopes_and_foothills,49.6,71.9,12.3,194.4,3395.5,86.6
northern_rockies,47.6,71.1,11.8,151.3,2354.3,64.8
puget_lowland,38.3,90.7,12.7,294.7,6154.1,177.8
willamette_valley,81.4,113.0,19.0,328.0,9516.5,242.7


,LIDAR_COVER,TOPHT,QMD,SDI,TCUFT,ABOVEGROUND_BIOMASS
count,7708.000000,7708.000000,7708.000000,7708.000000,7708.000000,7708.000000
mean,66.853895,96.780747,14.296492,315.817592,7410.013882,181.553321
std,30.918710,39.322274,8.100335,177.210972,6312.855165,121.967801
min,0.000000,6.000000,1.427506,2.000000,1.000000,2.000000
25%,49.677923,68.000000,8.730670,181.000000,2579.750000,88.000000
50%,80.035060,94.000000,12.820993,310.000000,5871.500000,165.000000
75%,91.370873,123.000000,17.797910,435.000000,10565.000000,246.000000
max,100.000000,267.000000,82.039021,1307.000000,51330.000000,887.000000


In [23]:
df = df.reset_index(drop=True)
X, Y = df[X_COLS], df[Y_COLS]

In [24]:
df[X_COLS].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7708 entries, 0 to 7707
Data columns (total 57 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   S2_R_LEAFOFF           7708 non-null   float64
 1   S2_G_LEAFOFF           7708 non-null   float64
 2   S2_B_LEAFOFF           7708 non-null   float64
 3   S2_NIR_LEAFOFF         7708 non-null   float64
 4   S2_SWIR1_LEAFOFF       7708 non-null   float64
 5   S2_SWIR2_LEAFOFF       7708 non-null   float64
 6   S2_RE1_LEAFOFF         7708 non-null   float64
 7   S2_RE2_LEAFOFF         7708 non-null   float64
 8   S2_RE3_LEAFOFF         7708 non-null   float64
 9   S2_RE4_LEAFOFF         7708 non-null   float64
 10  S2_R_LEAFON            7708 non-null   float64
 11  S2_G_LEAFON            7708 non-null   float64
 12  S2_B_LEAFON            7708 non-null   float64
 13  S2_NIR_LEAFON          7708 non-null   float64
 14  S2_SWIR1_LEAFON        7708 non-null   float64
 15  S2_S

In [25]:
df[Y_COLS].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7708 entries, 0 to 7707
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   LIDAR_COVER          7708 non-null   float64
 1   TOPHT                7708 non-null   int64  
 2   QMD                  7708 non-null   float64
 3   SDI                  7708 non-null   int64  
 4   TCUFT                7708 non-null   int64  
 5   ABOVEGROUND_BIOMASS  7708 non-null   int64  
dtypes: float64(2), int64(4)
memory usage: 361.4 KB


## Split datasets by ecoregion
We want to explore model transferability between regions, so we'll train models independently on subsets of the data within a single ecoregion, as well as a model that is trained on all available ecoregions. 

In [26]:
ecoregions = list(np.sort([reg for reg in pd.unique(df.ECOREGION3) if ecoreg_counts.loc[reg]['UUID'] > 20]))

eco_X_idx = [X.loc[X.ECOREGION3 == eco].index.values for eco in ecoregions]

eco_X_dfs = [X.loc[X.ECOREGION3 == eco].drop(['ECOREGION3'], axis=1) for eco in ecoregions]
eco_Y_dfs = [Y.loc[idx] for idx in eco_X_idx]

# append a "global" model that contains data from all ecoregions
ecoregions.append('all')
ecoregion_names = ['_'.join(x.split('_')[0:2]) for x in ecoregions]
eco_X_dfs.append(X.drop(['ECOREGION3'], axis=1))
eco_Y_dfs.append(Y)

ecoregion_display_names = [' '.join(x.upper().split('_')[:2]) for x in ecoregions]

In [27]:
cover_class_bins = [10,40,70,100]
cover_class_labels = ['OPEN', 'MODERATE', 'CLOSED']
height_class_bins = np.arange(0,300,20)
height_class_labels = [f'{x}-{x+20}' for x in height_class_bins[:-1]]
diameter_class_bins = [1, 5, 10, 15, 20, 999]
diameter_class_labels = ['SEED/SAP', 'SMALL', 'MEDIUM', 'LARGE', 'VERY_LARGE']

## Scoring
We'll use Root Mean Square Error to evaluate model performance.

In [28]:
def rmse(obs, pred):
    return np.sqrt((np.square(obs-pred)).mean())

def nrmse(obs, pred):
    return rmse(pred,obs) / obs.mean()

def mae(obs, pred):   
    return abs(pred - obs).mean()

def mape(obs, pred):    
    return abs(pred - obs).mean() / obs.mean()

def bias(obs, pred):   
    return (pred - obs).mean()

def rel_bias(obs, pred):
    return bias(pred,obs) / obs.mean()

def bin_accuracy(obs, pred, bins, fuzzy_tol=0):
    pred_binned = np.digitize(pred, bins)
    obs_binned = np.digitisze(obs, bins)
    diff = abs(pred_binned - obs_binned)
    
    return (diff <= fuzzy_tol).sum() / len(diff)

def confidence_interval_half(X, confidence=0.95):
    n = len(X)
    se = stats.sem(X)
    h = se * stats.t.ppf((1 + confidence) / 2., n-1)
    return h

This helper function will calculate RMSE scores for each regionally-trained model and the global model on each ecoregion.

## Fit some models
For each type of model, we'll employ cross-validation to tune model hyperparameters, generating a tuned model for each ecoregion as well as a tuned model using all training data. 

In [29]:
MODELS = {
    'ElasticNet': ElasticNet(selection='random'),
    'Lasso': Lasso(), 
    'KNeighborsRegressor': KNeighborsRegressor(n_jobs=-1),
    'RandomForestRegressor': RandomForestRegressor(n_jobs=-1), 
    'HistGradientBoostingRegressor': HistGradientBoostingRegressor(), 
}

FIT_PARAMS = {
    'ElasticNet': {
        'alpha': np.logspace(-4,2,7),
        'l1_ratio': np.arange(0.1, 1.0, 0.1),
    },
    'Lasso': {
        'alpha': np.logspace(-4,2,7),
    },
    'KNeighborsRegressor': {
        'n_neighbors': [1,2,3,4,5,10,20],
        'weights': ['uniform', 'distance'],
        'metric': ['minkowski', 'manhattan']
    },
    'RandomForestRegressor': {
        'n_estimators': [100, 500, 1000],
        'max_features': ['sqrt', None],
        'max_depth': [5, 20, None],
        'max_samples': [0.5, None]
    },
    'HistGradientBoostingRegressor': {
        'max_iter': [50, 100, 200],
        'min_samples_leaf': [5, 10, 20],
        'max_depth': [3, 5, 10],
        'learning_rate': [0.01, 0.1],
    },
}

In [30]:
NUM_OUTER_FOLDS = 5
NUM_INNER_FOLDS = 3
SCORE_FUNCS = [rmse, nrmse, mae, mape, bias, rel_bias]
score_names = [func.__name__ for func in SCORE_FUNCS]

In [31]:
def build_insider_results_dictionary(regions, model_names, num_outer_folds, score_funcs, target_vars):
    results = {}
    for region in regions:
        results[region] = {}
        for model_name in model_names:
            results[region][model_name] = {}
            results[region][model_name]['fitted_model'] = None
            results[region][model_name]['best_params'] = None
            results[region][model_name]['cv_results'] = {}
            for fold_idx in range(num_outer_folds):  # results from each outer loop of nested CV
                fold_num = fold_idx + 1
                results[region][model_name]['cv_results'][fold_num] = {}
                results[region][model_name]['cv_results'][fold_num]['best_params'] = None 
                results[region][model_name]['cv_results'][fold_num]['predict_time'] = None
                for score_func in score_funcs:
                    score_func_name = score_func.__name__
                    results[region][model_name]['cv_results'][fold_num][score_func_name] = {
                        y: None for y in target_vars
                    }
    return results

def build_global_results_dictionary(regions, model_names, num_outer_folds, score_funcs, target_vars):
    results = {}
    for model_name in model_names:
        results[model_name] = {}
        results[model_name]['fitted_model'] = None
        results[model_name]['best_params'] = None
        results[model_name]['cv_results'] = {}
        for fold_idx in range(num_outer_folds):  # results from each outer loop of nested CV
            fold_num = fold_idx + 1
            results[model_name]['cv_results'][fold_num] = {}
            results[model_name]['cv_results'][fold_num]['best_params'] = None 
            results[model_name]['cv_results'][fold_num]['predict_time'] = None
            for region in regions:
                results[model_name]['cv_results'][fold_num][region] = {}
                for score_func in score_funcs:
                    score_func_name = score_func.__name__
                    results[model_name]['cv_results'][fold_num][region][score_func_name] = {
                        y: None for y in target_vars
                    }
    return results

def build_outsider_results_dictionary(regions, model_names, score_funcs, target_vars):
    results = {}
    for region in regions:
        results[region] = {}
        for model_name in model_names:
            results[region][model_name] = {}
            results[region][model_name]['fitted_model'] = None
            results[region][model_name]['best_params'] = None
            results[region][model_name]['predict_time'] = None
            for score_func in score_funcs:
                score_func_name = score_func.__name__
                results[region][model_name][score_func_name] = {
                    y: None for y in target_vars
                }
    return results

def build_visiting_insider_results_dictionary(regions, model_names, score_funcs, target_vars):
    results = {}
    for target_region in regions:
        results[target_region] = {}
        for train_region in [r for r in regions if r != target_region]:
            results[target_region][train_region] = {}
            for model_name in model_names:
                results[target_region][train_region][model_name] = {}
                for score_func in score_funcs:
                    score_func_name = score_func.__name__
                    results[target_region][train_region][model_name][score_func_name] = {
                        y: None for y in target_vars
                    }
    return results

In [32]:
OUT_MODELS = '../models/structure_models2023'
os.makedirs(OUT_MODELS, exist_ok=True)

In [33]:
def tune_insider_model(model_name, num_outer_folds=NUM_OUTER_FOLDS, num_inner_folds=NUM_INNER_FOLDS):
    print(model_name)
    print('-'*len(model_name))
    model = MODELS[model_name]
    fit_params = FIT_PARAMS[model_name]
    train_regions = [x for x in ecoregions if x.upper() != 'ALL']
    
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('model', RegressorChain(base_estimator=model)),
    ])
    search_params = {f'model__base_estimator__{key}': value for key, value in fit_params.items()}
    
    cv_outer = GroupKFold(num_outer_folds)
    cv_inner = GroupKFold(num_inner_folds)
    
    for i, ecoregion in enumerate(train_regions):
        ecoregion_name = ecoregion_display_names[i]
        print(f'Starting on {ecoregion_name}', end='... ')
        X = eco_X_dfs[i]
        Y = eco_Y_dfs[i]
        outer_groups = df.loc[X.index, 'UUID'].values
        
        outer_fold_num = 1
        for train_ix, test_ix in cv_outer.split(X, groups=outer_groups):
            X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
            Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
            inner_groups = df.loc[X_train.index, 'UUID'].values
            
            inner_search = GridSearchCV(pipe, search_params, 
                                        scoring='neg_mean_squared_error', 
                                        n_jobs=-1, cv=cv_inner, refit=True)
            
            inner_result = inner_search.fit(X_train, Y_train, groups=inner_groups)
            insider_results[ecoregion][model_name]['cv_results'][outer_fold_num]['best_params'] = inner_result.best_params_
            
            inner_best_model = inner_result.best_estimator_
            start_time = time.time()
            Y_pred = inner_best_model.predict(X_test)
            end_time = time.time()
            total_predict_time = end_time - start_time
            avg_predict_time = total_predict_time / len(X_test)
            insider_results[ecoregion][model_name]['cv_results'][outer_fold_num]['predict_time'] = avg_predict_time
            
            for score_func in SCORE_FUNCS:
                score_func_name = score_func.__name__
                scores = score_func(Y_test, Y_pred)
                for y_var in scores.index:
                    insider_results[ecoregion][model_name]['cv_results'][outer_fold_num][score_func_name][y_var] = scores.loc[y_var]
                    
            print(outer_fold_num, end='... ')
            outer_fold_num += 1
        print('Done scoring. Now fitting a final model', end='... ')
        
        # done with scoring of models, now time to tune a model using the whole dataset
        outer_search = GridSearchCV(pipe, search_params, 
                                    scoring='neg_mean_squared_error', 
                                    n_jobs=-1, cv=cv_outer, refit=True)
        outer_result = outer_search.fit(X, Y, groups=outer_groups)
        
        # now fit on the entire dataset, not just training set
        model = outer_result.best_estimator_
        model.set_params(**outer_result.best_params_)
        X = df.loc[df.ECOREGION3 == ecoregion, X_COLS].drop(['ECOREGION3'], axis=1)
        y = df.loc[df.ECOREGION3 == ecoregion, Y_COLS]
        model.fit(X, y)

        eco_name = '_'.join(ecoregion.split('_')[:2])
        outfile = f'{eco_name}-sentinel-{model_name}-chained.pkl'
        outpath = os.path.join(OUT_MODELS, outfile)
        with open(outpath, 'wb') as file:
            pickle.dump(model, file)
        
        insider_results[ecoregion][model_name]['fitted_model'] = model
        insider_results[ecoregion][model_name]['best_params'] = outer_result.best_params_
        
        cv_results_dict = {ecoregion: insider_results[ecoregion][model_name]['cv_results'] for ecoregion in train_regions}
        print('All done.')
    
    return cv_results_dict

def tune_outsider_model(model_name, num_folds=5):
    print(model_name)
    print('-'*len(model_name))
    model = MODELS[model_name]
    fit_params = FIT_PARAMS[model_name]
    train_regions = [x for x in ecoregions if x.upper() != 'ALL']
    
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('model', RegressorChain(base_estimator=model)),
    ])
    search_params = {f'model__base_estimator__{key}': value for key, value in fit_params.items()}
    
    groupkfold = GroupKFold(num_folds)
    
    for i, ecoregion in enumerate(train_regions):
        ecoregion_name = ecoregion_display_names[i]
        print(f'Starting on {ecoregion_name}', end='... ')
        X_train = X.loc[X.ECOREGION3 != ecoregion].drop('ECOREGION3', axis=1)
        Y_train = Y.loc[X_train.index]
        X_test = X.loc[X.ECOREGION3 == ecoregion].drop('ECOREGION3', axis=1)
        Y_test = Y.loc[X_test.index]
        groups = df.loc[X_train.index]['ECOREGION3'].values
        
        search = GridSearchCV(pipe, search_params, 
                              scoring='neg_mean_squared_error',
                              n_jobs=-1, cv=groupkfold, refit=True)
            
        result = search.fit(X_train, Y_train, groups=groups)
        print('Done fitting, now scoring', end='... ')
        outsider_results[ecoregion][model_name]['best_params'] = result.best_params_
        outsider_results[ecoregion][model_name]['fitted_model'] = result.best_estimator_
        
        best_model = result.best_estimator_       
        start_time = time.time()
        Y_pred = best_model.predict(X_test)
        end_time = time.time()
        total_predict_time = end_time - start_time
        avg_predict_time = total_predict_time / len(X_test)
        outsider_results[ecoregion][model_name]['predict_time'] = avg_predict_time
            
        for score_func in SCORE_FUNCS:
            score_func_name = score_func.__name__
            scores = score_func(Y_test, Y_pred)
            for y_var in scores.index:
                outsider_results[ecoregion][model_name][score_func_name][y_var] = scores.loc[y_var]
        
        results_dict = {ecoregion: outsider_results[ecoregion][model_name] for ecoregion in train_regions}
        print('All done.')
    
    return results_dict

def tune_global_model(model_name, num_outer_folds=NUM_OUTER_FOLDS, num_inner_folds=NUM_INNER_FOLDS):
    print(model_name)
    print('-'*len(model_name))
    print(f'Scoring with {NUM_OUTER_FOLDS} folds... ', end='')
    model = MODELS[model_name]
    fit_params = FIT_PARAMS[model_name]
    test_regions = [x for x in ecoregions if x.upper() != 'ALL']
    
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('model', RegressorChain(base_estimator=model)),
    ])
    search_params = {f'model__base_estimator__{key}': value for key, value in fit_params.items()}
    
    cv_outer = GroupKFold(num_outer_folds)
    cv_inner = GroupKFold(num_inner_folds)
    
    X = df[X_COLS].drop('ECOREGION3', axis=1)
    Y = df[Y_COLS]
    outer_groups = df['UUID'].values
        
    outer_fold_num = 1
    for train_ix, test_ix in cv_outer.split(X, groups=outer_groups):
        X_train, X_test = X.loc[train_ix], X.loc[test_ix]
        Y_train, Y_test = Y.loc[train_ix], Y.loc[test_ix]
        inner_groups = df.loc[train_ix, 'UUID'].values

        inner_search = GridSearchCV(pipe, search_params, 
                                    scoring='neg_mean_squared_error', 
                                    n_jobs=-1, cv=cv_inner, refit=True)

        inner_result = inner_search.fit(X_train, Y_train, groups=inner_groups)
        global_results[model_name]['cv_results'][outer_fold_num]['best_params'] = inner_result.best_params_

        inner_best_model = inner_result.best_estimator_
        start_time = time.time()
        Y_pred = inner_best_model.predict(X_test)
        end_time = time.time()
        total_predict_time = end_time - start_time
        avg_predict_time = total_predict_time / len(X_test)
        global_results[model_name]['cv_results'][outer_fold_num]['predict_time'] = avg_predict_time

        for ecoregion in test_regions:
            region_mask = (df.loc[test_ix, 'ECOREGION3'] == ecoregion).values
            regional_X_test = X_test.loc[test_ix[region_mask]]
            regional_Y_test = Y_test.loc[test_ix[region_mask]]
            regional_Y_pred = inner_best_model.predict(regional_X_test)
            
            for score_func in SCORE_FUNCS:
                score_func_name = score_func.__name__
                scores = score_func(regional_Y_test, regional_Y_pred)
                for y_var in scores.index:
                    global_results[model_name]['cv_results'][outer_fold_num][ecoregion][score_func_name][y_var] = scores.loc[y_var]

        print(outer_fold_num, end='... ')
        outer_fold_num += 1
    
    print('Done scoring. Now fitting a final model', end='... ')
        
    # done with scoring of models, now time to tune a model using the whole dataset
    outer_search = GridSearchCV(pipe, search_params, 
                                scoring='neg_mean_squared_error', 
                                n_jobs=-1, cv=cv_outer, refit=True)
    outer_result = outer_search.fit(X, Y, groups=outer_groups)

     # now fit on the entire dataset, not just training set
    model = outer_result.best_estimator_
    model.set_params(**outer_result.best_params_)
    X = df[X_COLS].drop(['ECOREGION3'], axis=1)
    y = df[Y_COLS]
    model.fit(X, y)

    outfile = f'global-sentinel-{model_name}-chained.pkl'
    outpath = os.path.join(OUT_MODELS, outfile)
    with open(outpath, 'wb') as file:
        pickle.dump(model, file)
    print('All done.')

    global_results[model_name]['fitted_model'] = outer_result.best_estimator_
    global_results[model_name]['best_params'] = outer_result.best_params_

    results_dict = global_results[model_name]
    
    return results_dict

## Fit Global Models
These models get to see data from every ecoregion during training and tuning.

In [34]:
global_results = build_global_results_dictionary(ecoregions[:-1], MODELS.keys(), NUM_OUTER_FOLDS, SCORE_FUNCS, Y_COLS)

In [35]:
def parse_global_results(results):
    data = []
    for fold in range(NUM_OUTER_FOLDS):
        for ecoregion in ecoregions[:-1]:
            for target in Y_COLS:
                for score_name in score_names:
                    data.append((fold+1, ecoregion, target, score_name, results['cv_results'][fold+1][ecoregion][score_name][target]))
    return pd.DataFrame(data, columns=['cv_fold', 'ecoregion', 'target', 'metric', 'score'])

In [ ]:
os.environ["PYTHONWARNINGS"] = "ignore"
elastic_global = tune_global_model('ElasticNet')
os.environ["PYTHONWARNINGS"] = ""
lasso_global = tune_global_model('Lasso')
knn_global = tune_global_model('KNeighborsRegressor')
rf_global = tune_global_model('RandomForestRegressor')
gbm_global = tune_global_model('HistGradientBoostingRegressor')

ElasticNet
----------
Scoring with 5 folds... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
Lasso
-----
Scoring with 5 folds... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
KNeighborsRegressor
-------------------
Scoring with 5 folds... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
RandomForestRegressor
---------------------
Scoring with 5 folds... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... 

In [ ]:
RESULTS_TO_CONCAT = [elastic_global, lasso_global, knn_global, rf_global, gbm_global]
NAMES = ['ElasticNet', 'Lasso', 'kNN', 'RF', 'GBM']
dfs_to_concat = []
for res, name in zip(RESULTS_TO_CONCAT, NAMES):
    tmp_df = parse_global_results(res)
    tmp_df['model'] = name
    dfs_to_concat.append(tmp_df)
all_global_results = pd.concat(dfs_to_concat, axis=0, ignore_index=True)
all_global_results['ecoregion'] = all_global_results['ecoregion'].apply(lambda x: ' '.join(x.title().replace('_',' ').split()[:2]))
all_global_results.columns = [col.upper() for col in all_global_results.columns]
# all_global_results = all_global_results.rename({'SCORE': 'MAE'}, axis=1)
all_global_results.head()

In [ ]:
all_global_results.to_csv('../data/processed/nestedcv_chained_global_results_satellite_structure_lidar_cover.csv', header=True, index=False)

## Fit Outsider Models
These models have data from the ecoregion they're tested on held out during training.

In [34]:
outsider_results = build_outsider_results_dictionary(ecoregions[:-1], MODELS.keys(), SCORE_FUNCS, Y_COLS)

In [35]:
def parse_outsider_results(results):
    data = []
    for ecoregion in ecoregions[:-1]:
        for target in Y_COLS:
            for score_name in score_names:
                data.append((np.nan, ecoregion, target, score_name, results[ecoregion][score_name][target]))
    return pd.DataFrame(data, columns=['cv_fold', 'ecoregion', 'target', 'metric', 'score'])

In [36]:
os.environ["PYTHONWARNINGS"] = "ignore"
elastic_outsider = tune_outsider_model('ElasticNet')
os.environ["PYTHONWARNINGS"] = ""
lasso_outsider = tune_outsider_model('Lasso')
knn_outsider = tune_outsider_model('KNeighborsRegressor')
rf_outsider = tune_outsider_model('RandomForestRegressor')
gbm_outsider = tune_outsider_model('HistGradientBoostingRegressor')

ElasticNet
----------
Starting on BLUE MOUNTAINS... Done fitting, now scoring... All done.
Starting on CASCADES... Done fitting, now scoring... All done.
Starting on COAST RANGE... Done fitting, now scoring... All done.
Starting on EASTERN CASCADES... Done fitting, now scoring... All done.
Starting on KLAMATH MOUNTAINS... Done fitting, now scoring... All done.
Starting on NORTH CASCADES... Done fitting, now scoring... All done.
Starting on NORTHERN ROCKIES... Done fitting, now scoring... All done.
Starting on PUGET LOWLAND... Done fitting, now scoring... All done.
Starting on WILLAMETTE VALLEY... Done fitting, now scoring... All done.
Lasso
-----
Starting on BLUE MOUNTAINS... Done fitting, now scoring... All done.
Starting on CASCADES... Done fitting, now scoring... All done.
Starting on COAST RANGE... Done fitting, now scoring... All done.
Starting on EASTERN CASCADES... Done fitting, now scoring... All done.
Starting on KLAMATH MOUNTAINS... Done fitting, now scoring... All done.
Star

In [37]:
RESULTS_TO_CONCAT = [elastic_outsider, lasso_outsider, knn_outsider, rf_outsider, gbm_outsider]
NAMES = ['ElasticNet', 'Lasso', 'kNN', 'RF', 'GBM']
dfs_to_concat = []
for res, name in zip(RESULTS_TO_CONCAT, NAMES):
    tmp_df = parse_outsider_results(res)
    tmp_df['model'] = name
    dfs_to_concat.append(tmp_df)
all_outsider_results = pd.concat(dfs_to_concat, axis=0, ignore_index=True)
all_outsider_results['ecoregion'] = all_outsider_results['ecoregion'].apply(lambda x: ' '.join(x.title().replace('_',' ').split()[:2]))
all_outsider_results.columns = [col.upper() for col in all_outsider_results.columns]
all_outsider_results.head()

,CV_FOLD,ECOREGION,TARGET,METRIC,SCORE,MODEL
0,NaN,Blue Mountains,LIDAR_COVER,rmse,16.072562,ElasticNet
1,NaN,Blue Mountains,LIDAR_COVER,nrmse,0.486345,ElasticNet
2,NaN,Blue Mountains,LIDAR_COVER,mae,12.739182,ElasticNet
3,NaN,Blue Mountains,LIDAR_COVER,mape,0.385479,ElasticNet
4,NaN,Blue Mountains,LIDAR_COVER,bias,10.434625,ElasticNet


In [38]:
all_outsider_results.to_csv('../data/processed/nestedcv_chained_outsider_results_satellite_structure_lidar_cover.csv', header=True, index=False)

## Fit Insider Models
These models are trained with observations from a single ecoregion.

In [42]:
insider_results = build_insider_results_dictionary(ecoregions[:-1], MODELS.keys(), 5, SCORE_FUNCS, Y_COLS)

In [43]:
def parse_insider_results(results):
    data = []
    for ecoregion in ecoregions[:-1]:
        for fold_num in results[ecoregion].keys():
            for target in Y_COLS:
                for score_name in score_names:
                    data.append((fold_num, ecoregion, target, score_name, results[ecoregion][fold_num][score_name][target]))
    return pd.DataFrame(data, columns=['cv_fold', 'ecoregion', 'target', 'metric', 'score'])

In [44]:
os.environ["PYTHONWARNINGS"] = "ignore"
elastic_insider = tune_insider_model('ElasticNet')
os.environ["PYTHONWARNINGS"] = ""
lasso_insider = tune_insider_model('Lasso')
knn_insider = tune_insider_model('KNeighborsRegressor')
rf_insider = tune_insider_model('RandomForestRegressor')
gbm_insider = tune_insider_model('HistGradientBoostingRegressor')

ElasticNet
----------
Starting on BLUE MOUNTAINS... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
Starting on CASCADES... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
Starting on COAST RANGE... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
Starting on EASTERN CASCADES... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
Starting on KLAMATH MOUNTAINS... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
Starting on NORTH CASCADES... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
Starting on NORTHERN ROCKIES... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
Starting on PUGET LOWLAND... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
Starting on WILLAMETTE VALLEY... 1... 2... 3... 4... 5... Done scoring. Now fitting a final model... All done.
Lasso


In [45]:
RESULTS_TO_CONCAT = [elastic_insider, lasso_insider, knn_insider, rf_insider, gbm_insider]
NAMES = ['ElasticNet', 'Lasso', 'kNN', 'RF', 'GBM']
dfs_to_concat = []
for res, name in zip(RESULTS_TO_CONCAT, NAMES):
    tmp_df = parse_insider_results(res)
    tmp_df['model'] = name
    dfs_to_concat.append(tmp_df)
all_insider_results = pd.concat(dfs_to_concat, axis=0, ignore_index=True)
all_insider_results['ecoregion'] = all_insider_results['ecoregion'].apply(lambda x: ' '.join(x.title().replace('_',' ').split()[:2]))
all_insider_results.columns = [col.upper() for col in all_insider_results.columns]
all_insider_results.head()

,CV_FOLD,ECOREGION,TARGET,METRIC,SCORE,MODEL
0,1,Blue Mountains,LIDAR_COVER,rmse,9.404883,ElasticNet
1,1,Blue Mountains,LIDAR_COVER,nrmse,0.276058,ElasticNet
2,1,Blue Mountains,LIDAR_COVER,mae,7.190930,ElasticNet
3,1,Blue Mountains,LIDAR_COVER,mape,0.211072,ElasticNet
4,1,Blue Mountains,LIDAR_COVER,bias,2.102870,ElasticNet


In [46]:
all_insider_results.to_csv('../data/processed/nestedcv_chained_insider_results_satellite_structure_lidar_cover.csv', header=True, index=False)

## Use Trained Insider Models to Score Visiting Insider Models
These models are trained on a single region, and scored on other regions they've never seen before. 

In [47]:
visitor_results = build_visiting_insider_results_dictionary(ecoregions[:-1], MODELS.keys(), SCORE_FUNCS, Y_COLS)

In [48]:
visitor_results = []
for target_region in ecoregions[:-1]:
    for train_region in [r for r in ecoregions[:-1] if r != target_region]:
        for model_name in MODELS.keys():
            model = insider_results[train_region][model_name]['fitted_model']
            targ_idx = X.loc[X.ECOREGION3 == target_region].index.values
            targ_X = X.loc[targ_idx].drop(['ECOREGION3'], axis=1)
            pred = model.predict(targ_X)
            obs = Y.loc[targ_idx]
            for score_func in SCORE_FUNCS:
                score_func_name = score_func.__name__
                scores = score_func(obs, pred)
                for y, score in scores.iteritems():
                    visitor_results.append(
                        (' '.join(target_region.title().replace('_',' ').split()),
                         ' '.join(train_region.title().replace('_',' ').split()),
                         model_name, score_func_name, y, score))
visitor_df = pd.DataFrame(visitor_results, 
                          columns = ['TARGET_ECOREGION', 'TRAIN_ECOREGION', 
                                     'MODEL', 'METRIC', 'TARGET', 'SCORE'])
visitor_df.head()

,TARGET_ECOREGION,TRAIN_ECOREGION,MODEL,METRIC,TARGET,SCORE
0,Blue Mountains,Cascades,ElasticNet,rmse,LIDAR_COVER,27.573275
1,Blue Mountains,Cascades,ElasticNet,rmse,TOPHT,33.872656
2,Blue Mountains,Cascades,ElasticNet,rmse,QMD,16.857373
3,Blue Mountains,Cascades,ElasticNet,rmse,SDI,216.018045
4,Blue Mountains,Cascades,ElasticNet,rmse,TCUFT,5116.483351


In [49]:
visitor_df.to_csv('../data/processed/nestedcv_chained_visitor_results_satellite_structure_lidar_cover.csv', 
                  header=True, index=False)